In [1]:
import pandas as pd
import numpy as np
import nltk
import math
import time
import operator
import string
import csv
from sklearn.cluster import KMeans
from math import pi
from sklearn.metrics import mean_squared_error
from nltk.corpus import wordnet, stopwords, sentiwordnet as swn
from nltk.tokenize import sent_tokenize, word_tokenize, WhitespaceTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
f = open('train_data.csv','r', errors='ignore')
train_data = pd.read_csv(f,
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True)
length_train = len(train_data.review_text)

f1 = open('test_data.csv','r', errors='ignore')
test_data = pd.read_csv(f1,
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True)
train_data.head()


,asin,date,downvotes,review_comments,review_id,review_text,review_type,reviewer,star_rating,upvotes,...,in_app_purchase,mas_rating,min_os_version,num_permissions,num_screenshots,price,release_date,size_MB,title,update_date
0,B00OPRXVIC,2014-11-06,NaN,0,RAC9Q5IDVW1TC,this is cool. parents can write an email from ...,Verified Purchase,A2NGN1TFABVU8Q,5,NaN,...,0,Guidance Suggested,2.3.3,1,10,199.0,2014-10-21,0.32041,Email From Santa 2014,2014-11-06
1,B008JQPY2G,2012-07-21,NaN,0,R295U1EYQDU4CN,its no good I didnt like it at all,Verified Purchase,A3SHL50ZU1K96M,1,NaN,...,0,All Ages,2.1,2,4,99.0,2012-07-11,12.80000,Monsters - Difference Games - Game App,2012-07-11
2,B008JQPY2G,2013-01-04,NaN,0,R2Z8UUF4V7ERKX,absolutely terrible. I mean that I did not lik...,NaN,A9MOKV997YYIN,1,NaN,...,0,All Ages,2.1,2,4,99.0,2012-07-11,12.80000,Monsters - Difference Games - Game App,2012-07-11
3,B008JQPY2G,2015-10-16,NaN,0,R35VWHNI4UXNIP,Fun game,Verified Purchase,A3VYN5P1IRUL3H,5,NaN,...,0,All Ages,2.1,2,4,99.0,2012-07-11,12.80000,Monsters - Difference Games - Game App,2012-07-11
4,B008JQPY2G,2013-03-05,NaN,0,R7LQK4NGD2ATF,"This game downloaded for me, but when I try to...",Verified Purchase,AIX9ODIJLREIM,1,NaN,...,0,All Ages,2.1,2,4,99.0,2012-07-11,12.80000,Monsters - Difference Games - Game App,2012-07-11


In [12]:
train_review = pd.Series.tolist(train_data.review_text)
train_rating = pd.Series.tolist(train_data.star_rating)
train_model = [None]*5
for i in range(5):
    train_model[i] = [str(tmp) for tmp, score in zip(train_review, train_rating) if score == i+1]

In [13]:
most_word = [None]*5
feature = [None]*5
feature_learn = [None]*5
test_model = [None]*5
test_review = pd.Series.tolist(test_data.review_text)
test_review = [str(tmp) for tmp in test_review]
#train_review = [str(tmp) for tmp in train_review]
voc_model = TfidfVectorizer(decode_error='ignore', ngram_range=(1,2), max_df=0.8, min_df=0.01, stop_words='english')
#voc_tf_model = CountVectorizer(decode_error='ignore', ngram_range=(1,2), stop_words='english')
for i in range(5):
    feature[i] = voc_model.fit_transform(train_model[i])
    #most_word[i] = voc_model.get_feature_names()
    feature_learn[i] = voc_model.transform(train_model[i])
    test_model[i] = voc_model.transform(test_review)

In [50]:
# for i in range(5):
#     for words in most_word[i]:
#         syns = wordnet.synsets(words)
#         word_score_pos = 0
#         word_score_neg = 0
#         for t in range(len(syns)):
#             if word_score_pos < swn.senti_synset(syns[t].name()).pos_score():
#                 word_score_pos = swn.senti_synset(syns[t].name()).pos_score()
#             if word_score_neg < swn.senti_synset(syns[t].name()).neg_score():
#                 word_score_neg = swn.senti_synset(syns[t].name()).neg_score()
#         if word_score_pos == 0 and word_score_neg == 0:
#             most_word[i].remove(words)

In [14]:
kmeans = KMeans(n_clusters = 1, init='k-means++')
centroids = [None]*5
for i in range(5):
    kmeans.fit(feature_learn[i])
    centroids[i] = kmeans.cluster_centers_

In [17]:
distance = np.zeros((5,100000))
for i in range(5):
    for j in range(100000):
        distance[i][j] = mean_squared_error(centroids[i], test_model[i][j].toarray())

In [18]:
prob = np.zeros((100000,5))
distance_t = np.transpose(distance)
for i in range(100000):
    for j in range(5):
        prob[i][j] = distance_t[i][j]/sum(distance_t[i])

In [452]:
prob1 = prob.astype('str')
review_id = pd.Series.tolist(test_data.review_id)
review_id = np.asarray([[str(tmp)] for tmp in review_id])

In [453]:
result = np.append(review_id, prob1, 1)
title = np.asarray([['review_id','1-star','2-star','3-star','4-star','5-star']])
result = np.append(title,result,0)
#result = [np.array(tmp) for tmp in result]
result = result.tolist()
result = np.asarray(result)
np.savetxt("result.csv",result,fmt='%s,%s,%s,%s,%s,%s')